<a href="https://colab.research.google.com/github/iremaricii/Cat_or_Dog_Classification/blob/main/ocr_project_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

tf.get_logger().setLevel('ERROR')

# Sabitler ve ayarlar
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
target_size = (28, 28)      # MNIST görüntü boyutu
batch_size = 32
num_classes = 10            # 0-9 rakamlar

##############################################
# 1. Veri Setlerinin Oluşturulması
##############################################
# 1.1 Handwriting (El Yazısı) Verisi: MNIST
(x_train_hw, y_train_hw), (x_test_hw, y_test_hw) = tf.keras.datasets.mnist.load_data()

# 1.2 Typewriter (Daktilo) Verisinin Sentetik Olarak Oluşturulması
def generate_typewriter_image(label, target_size=(28,28)):
    """Belirtilen etiket için beyaz arkaplan üzerine OpenCV ile rakam yazar."""
    img = np.ones(target_size, dtype=np.uint8) * 255
    text = str(label)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    textX = (target_size[1] - text_size[0]) // 2
    textY = (target_size[0] + text_size[1]) // 2
    cv2.putText(img, text, (textX, textY), font, font_scale, (0,), thickness, cv2.LINE_AA)
    return img

x_train_tp = np.array([generate_typewriter_image(label, target_size) for label in y_train_hw])
y_train_tp = y_train_hw.copy()
x_test_tp = np.array([generate_typewriter_image(label, target_size) for label in y_test_hw])
y_test_tp = y_test_hw.copy()

##############################################
# 2. Eğitim, Doğrulama ve Test Kümelemelerinin Yapılması
##############################################
# Eğitim/validation bölünmesi (%70/%30) el yazısı ve daktilo için
x_train_hw_train, x_train_hw_val, y_train_hw_train, y_train_hw_val = train_test_split(
    x_train_hw, y_train_hw, test_size=0.3, random_state=seed)
x_train_tp_train, x_train_tp_val, y_train_tp_train, y_train_tp_val = train_test_split(
    x_train_tp, y_train_tp, test_size=0.3, random_state=seed)

# Test verisi
x_test_hw_final, y_test_hw_final = x_test_hw, y_test_hw
x_test_tp_final, y_test_tp_final = x_test_tp, y_test_tp

# Kanal boyutunu ekleme: (28,28) -> (28,28,1)
x_train_hw_train = x_train_hw_train[..., np.newaxis]
x_train_tp_train = x_train_tp_train[..., np.newaxis]
x_train_hw_val = x_train_hw_val[..., np.newaxis]
x_train_tp_val = x_train_tp_val[..., np.newaxis]
x_test_hw_final = x_test_hw_final[..., np.newaxis]
x_test_tp_final = x_test_tp_final[..., np.newaxis]

# Eğitim verilerini birleştirme
x_train_combined = np.concatenate([x_train_hw_train, x_train_tp_train], axis=0)
y_train_combined = np.concatenate([y_train_hw_train, y_train_tp_train], axis=0)
x_val_combined = np.concatenate([x_train_hw_val, x_train_tp_val], axis=0)
y_val_combined = np.concatenate([y_train_hw_val, y_train_tp_val], axis=0)
x_test_combined = np.concatenate([x_test_hw_final, x_test_tp_final], axis=0)
y_test_combined = np.concatenate([y_test_hw_final, y_test_tp_final], axis=0)

##############################################
# 3. Görüntü Ön İşleme: OpenCV İşlemleri
##############################################
def cv2_preprocess(image):
    """
    Gelen görüntü için:
      - Eğer Tensor ise numpy array'e çevirme,
      - Gerekirse kanal boyutunu ayarlama,
      - Gaussian blur, Otsu eşikleme ve Canny kenar tespiti uygulama,
      - Sonuç olarak 2 kanallı (eşik, kenar) görüntü üretme.
    """
    if isinstance(image, tf.Tensor):
        image = image.numpy()
    if image.ndim == 3 and image.shape[-1] == 1:
        image = np.squeeze(image, axis=-1)
    image_uint8 = np.uint8(image)
    blurred = cv2.GaussianBlur(image_uint8, (3, 3), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh_norm = thresh.astype('float32') / 255.0
    edges = cv2.Canny(blurred, 50, 150)
    edges_norm = edges.astype('float32') / 255.0
    thresh_norm = np.expand_dims(thresh_norm, axis=-1)
    edges_norm = np.expand_dims(edges_norm, axis=-1)
    combined = np.concatenate([thresh_norm, edges_norm], axis=-1)
    return combined

def preprocess_py(image, label):
    processed = tf.py_function(func=lambda x: cv2_preprocess(x), inp=[image], Tout=tf.float32)
    processed.set_shape((target_size[0], target_size[1], 2))
    return processed, label

##############################################
# 4. tf.data.Dataset Nesnelerinin Oluşturulması
##############################################
train_ds = tf.data.Dataset.from_tensor_slices((x_train_combined, y_train_combined))
train_ds = train_ds.map(preprocess_py, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_val_combined, y_val_combined))
val_ds = val_ds.map(preprocess_py, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test_combined, y_test_combined))
test_ds = test_ds.map(preprocess_py, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

##############################################
# 5. Modelin Tanımlanması ve Eğitilmesi
##############################################
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

input_shape_model = (target_size[0], target_size[1], 2)
model = build_model(input_shape_model, num_classes)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Eğitim: verbose=2 ile sadece her epoch sonunda özet çıktı alınır.
epochs = 5
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

##############################################
# 6. Eğitim Sonuçlarının Görselleştirilmesi
##############################################
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], marker='o', label='Train Loss')
plt.plot(history.history['val_loss'], marker='o', label='Val Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], marker='o', label='Train Accuracy')
plt.plot(history.history['val_accuracy'], marker='o', label='Val Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

##############################################
# 7. Genel Test Sonuçlarının Değerlendirilmesi
##############################################
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

y_true = []
y_pred = []
for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))
print("General Test Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[str(i) for i in range(num_classes)],
            yticklabels=[str(i) for i in range(num_classes)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - General Test")
plt.show()

##############################################
# 8. Ayrı Değerlendirme: Handwriting vs. Typewriter
##############################################
# Handwriting Test Seti
test_ds_hw = tf.data.Dataset.from_tensor_slices((x_test_hw_final, y_test_hw_final))
test_ds_hw = test_ds_hw.map(preprocess_py, num_parallel_calls=tf.data.AUTOTUNE)
test_ds_hw = test_ds_hw.batch(batch_size)
y_true_hw = []
y_pred_hw = []
for images, labels in test_ds_hw:
    preds = model.predict(images, verbose=0)
    y_true_hw.extend(labels.numpy())
    y_pred_hw.extend(np.argmax(preds, axis=1))
print("Handwriting Test Classification Report:")
print(classification_report(y_true_hw, y_pred_hw, digits=4))
cm_hw = confusion_matrix(y_true_hw, y_pred_hw)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_hw, annot=True, fmt='d', cmap='Greens',
            xticklabels=[str(i) for i in range(num_classes)],
            yticklabels=[str(i) for i in range(num_classes)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Handwriting Test")
plt.show()

# Typewriter Test Seti
test_ds_tp = tf.data.Dataset.from_tensor_slices((x_test_tp_final, y_test_tp_final))
test_ds_tp = test_ds_tp.map(preprocess_py, num_parallel_calls=tf.data.AUTOTUNE)
test_ds_tp = test_ds_tp.batch(batch_size)
y_true_tp = []
y_pred_tp = []
for images, labels in test_ds_tp:
    preds = model.predict(images, verbose=0)
    y_true_tp.extend(labels.numpy())
    y_pred_tp.extend(np.argmax(preds, axis=1))
print("Typewriter Test Classification Report:")
print(classification_report(y_true_tp, y_pred_tp, digits=4))
cm_tp = confusion_matrix(y_true_tp, y_pred_tp)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_tp, annot=True, fmt='d', cmap='Oranges',
            xticklabels=[str(i) for i in range(num_classes)],
            yticklabels=[str(i) for i in range(num_classes)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Typewriter Test")
plt.show()

##############################################
# 9. Ek Profesyonel Görselleştirmeler
##############################################
# 9.1 t-SNE Görselleştirmesi (Özellik Uzayı)
# Modelin ilk katmanının girişini kullanarak feature_model oluşturuyoruz.
feature_model = tf.keras.Model(inputs=model.layers[0].input, outputs=model.layers[5].output)
features = feature_model.predict(test_ds, verbose=0)
tsne = TSNE(n_components=2, random_state=seed, perplexity=30)
tsne_results = tsne.fit_transform(features)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=y_true, cmap='tab10', s=10)
plt.colorbar(scatter, ticks=range(num_classes))
plt.title("t-SNE Visualization - General Test Features")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.grid(True)
plt.show()

features_hw = feature_model.predict(test_ds_hw, verbose=0)
tsne_hw = TSNE(n_components=2, random_state=seed, perplexity=30)
tsne_results_hw = tsne_hw.fit_transform(features_hw)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(tsne_results_hw[:, 0], tsne_results_hw[:, 1], c=y_true_hw, cmap='tab10', s=10)
plt.colorbar(scatter, ticks=range(num_classes))
plt.title("t-SNE Visualization - Handwriting Test Features")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.grid(True)
plt.show()

features_tp = feature_model.predict(test_ds_tp, verbose=0)
tsne_tp = TSNE(n_components=2, random_state=seed, perplexity=30)
tsne_results_tp = tsne_tp.fit_transform(features_tp)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(tsne_results_tp[:, 0], tsne_results_tp[:, 1], c=y_true_tp, cmap='tab10', s=10)
plt.colorbar(scatter, ticks=range(num_classes))
plt.title("t-SNE Visualization - Typewriter Test Features")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.grid(True)
plt.show()

# 9.2 Sınıf Bazlı Metrik Bar Grafiği (Precision, Recall, F1)
report_dict = classification_report(y_true, y_pred, output_dict=True)
classes = [str(i) for i in range(num_classes)]
precision_vals = [report_dict[str(i)]['precision'] for i in range(num_classes)]
recall_vals = [report_dict[str(i)]['recall'] for i in range(num_classes)]
f1_vals = [report_dict[str(i)]['f1-score'] for i in range(num_classes)]
plt.figure(figsize=(10,6))
x_axis = np.arange(num_classes)
width = 0.25
plt.bar(x_axis - width, precision_vals, width, label='Precision')
plt.bar(x_axis, recall_vals, width, label='Recall')
plt.bar(x_axis + width, f1_vals, width, label='F1-score')
plt.xticks(x_axis, classes)
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Classification Metrics per Class')
plt.legend()
plt.grid(True)
plt.show()

# 9.3 Rastgele Test Örneklerinin Görselleştirilmesi
all_images = []
all_labels = []
for images, labels in test_ds:
    for i in range(images.shape[0]):
        all_images.append(images[i])
        all_labels.append(labels.numpy()[i])
all_images = np.array(all_images)
all_labels = np.array(all_labels)
preds = model.predict(test_ds, verbose=0)
all_preds = np.argmax(preds, axis=1)
n_display = 12
indices = np.random.choice(len(all_images), n_display, replace=False)
plt.figure(figsize=(12, 8))
for j, idx in enumerate(indices):
    img = all_images[idx].numpy() if isinstance(all_images[idx], tf.Tensor) else all_images[idx]
    img_disp = img[:, :, 0]
    true_label = all_labels[idx]
    pred_label = all_preds[idx]
    plt.subplot(3, 4, j+1)
    plt.imshow(img_disp, cmap='gray')
    plt.title(f"True: {true_label}, Pred: {pred_label}")
    plt.axis('off')
plt.suptitle("Random Test Examples")
plt.tight_layout()
plt.show()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,322 (880.16 KB)

 Trainable params: 225,322 (880.16 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
2625/2625 - 170s - 65ms/step - accuracy: 0.9521 - loss: 0.1581 - val_accuracy: 0.9607 - val_loss: 0.1646
Epoch 2/5
2625/2625 - 193s - 74ms/step - accuracy: 0.9828 - loss: 0.0594 - val_accuracy: 0.9906 - val_loss: 0.0355
Epoch 3/5
2625/2625 - 150s - 57ms/step - accuracy: 0.9880 - loss: 0.0410 - val_accuracy: 0.9919 - val_loss: 0.0323
Epoch 4/5
